# Fase 0.1: Convertir la CNN inicial de PyTorch a ONXX

In [3]:
# Imports

import math,random,struct,os,time,sys
import numpy as np
import torch
import torch.nn as nn
import torch.onnx
import onnx
import onnxruntime as ort
from torchvision import models
from torch.utils.data import Dataset,DataLoader
from sklearn import preprocessing
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import time

# Funciones y parámetros de la CNN base
import sys
sys.path.append('../src/')

from cnn21_pix import *

## 1. Conversión de la CNN inicial de PyTorch a ONNX

### 1.1. Conversión del modelo

In [6]:
# Datos de entrada
batch_size = 100
B = 5
sizex = 32
sizey = 32

In [7]:
# Convertir el modelo a ONNX

device='cpu'
model = torch.load("../results/models/model_cnn21.pth", weights_only=False)
model.eval()

# Crear un tensor de entrada de ejemplo 
# El tensor tendrá tamanho (batch_size, canales, altura, ancho)
input_tensor = torch.randn(batch_size, B, sizex, sizey).to(device)

# Exportamos el modelo a onnx
onnx_filename = "../results/models/model_cnn21.onnx"
torch.onnx.export(
    model, # Modelo entrenado
    input_tensor, # Entrada de ejemplo
    onnx_filename, # Ruta de salida del archivo ONNX
    export_params=True, # Exportar parámetros del modelo
    opset_version=12, # Versión de opset
    do_constant_folding=True, # Optimización de constantes
    input_names=['input'], # Nombre de la entrada
    output_names=['output'], # Nombre de la salida
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)

In [8]:
# Comprobar que la conversión es correcta

onnx_model = onnx.load(onnx_filename)
onnx.checker.check_model(onnx_model)

### 1.2. Evaluación del modelo en ONNX

In [9]:
# Cargar los modelos

# Cargar el modelo original de PyTorch
model = torch.load("../results/models/model_cnn21.pth", weights_only=False)
model.eval()

# Cargar el modelo convertido a ONNX
ort_session = ort.InferenceSession("../results/models/model_cnn21.onnx")

In [10]:
# Comprobar la diferencia de precisión

# Crear tensor de entrada de prueba
input_tensor = torch.randn(1, B, sizex, sizey).to(device)

# Salida del modelo PyTorch
with torch.no_grad():
    output_torch = model(input_tensor).cpu().numpy()

# Salida del modelo ONNX
output_onnx = ort_session.run(None, {'input': input_tensor.cpu().numpy()})[0]

# Comparar las diferencias
error = np.abs(output_torch - output_onnx).mean()
print(f'Error medio entre PyTorch y ONNX: {error}')

Error medio entre PyTorch y ONNX: 1.1444091796875e-05


In [ ]:
# Carga de datos e inferencia completa para el modelo en ONNX

# Medir el tiempo de ejecución
start_time = time.time()

# Definir parámetros y cargar datos

DATASET='../data/imagenes_rios/oitaven_river.raw'
GT='../data/imagenes_rios/oitaven_river.pgm'

# Queremos usar todos los datos para la inferencia
SAMPLES=[0,0]
PAD=1
AUM=0

# Carga de datos
(datos,H,V,B)=read_raw(DATASET)
(truth,H1,V1)=read_pgm(GT)

# Durante la ejecucion de la red vamos a coger patches de tamano cuadrado
sizex=32; sizey=32 

# Hacemos padding en el dataset para poder aprovechar hasta el borde
if(PAD):
    datos=torch.FloatTensor(np.pad(datos,((sizey//2,sizey//2),(sizex//2,sizex//2),(0,0)),'symmetric'))
    H=H+2*(sizex//2); V=V+2*(sizey//2)
    truth=np.reshape(truth,(-1,H1))
    truth=np.pad(truth,((sizey//2,sizey//2),(sizex//2,sizex//2)),'constant')
    H1=H1+2*(sizex//2); V1=V1+2*(sizey//2)
    truth=np.reshape(truth,(H1*V1))
    
# Necesitamos los datos en band-vector para hacer convoluciones
datos=np.transpose(datos,(2,0,1))

# Seleccionar conjunto de test (en este caso es una predicción)
(train,val,test,nclases,nclases_no_vacias)=select_training_samples(truth,H,V,sizex,sizey,SAMPLES)
dataset_test=HyperDataset(datos,truth,test,H,V,sizex,sizey)
print('  - test dataset:',len(dataset_test))

# Dataloader
batch_size=100 # defecto 100
test_loader=DataLoader(dataset_test,batch_size,shuffle=False)

output=np.zeros(H*V,dtype=np.uint8)

# Modo evaluación
model.eval()

# Realizar la predicción
total=0
for (inputs, labels) in test_loader:
    # Convertir inputs a un formato adecuado para ONNX (numpy array)
    inputs_np = inputs.numpy()
    
    # Realizar la inferencia
    outputs = ort_session.run(None, {'input': inputs_np})
    
    predicted=np.argmax(outputs[0], axis=1) # outputs[0] contiene las predicciones
    
    # Asignar las predicciones al array de salida
    for i in range(len(predicted)):
        output[test[total+i]]=np.uint8(predicted[i]+1)
    total+=labels.size(0)
    
    # Mostrar el progreso
    if(total%100000==0): print('  Test:',total,'/',len(dataset_test))

end_time = time.time()

print("Prediction time: {:.4f} seconds".format(end_time - start_time))

In [ ]:
# Guardar el output

np.save('../results/predictions/predictions_cnn21_onnx.npy', output)

In [ ]:
# Cargar el output

output = np.load('../results/predictions/predictions_cnn21_onnx.npy')

In [ ]:
# Evaluar el desempeño del modelo

# Precisiones a nivel de clase
correct=0; total=0; AA=0; OA=0
class_correct=[0]*(nclases+1)
class_total=[0]*(nclases+1)
class_aa=[0]*(nclases+1)

for i in test:
    if(output[i]==0 or truth[i]==0): continue
    total+=1; class_total[truth[i]]+=1
    if(output[i]==truth[i]):
          correct+=1
          class_correct[truth[i]]+=1
for i in range(1,nclases+1):
    if(class_total[i]!=0): class_aa[i]=100*class_correct[i]/class_total[i]
    else: class_aa[i]=0
    AA+=class_aa[i]
OA=100*correct/total; AA=AA/nclases_no_vacias

for i in range(1,nclases+1): print('  Class %02d: %02.02f'%(i,class_aa[i]))
print('* Accuracy (pixels) OA=%02.02f, AA=%02.02f'%(OA,AA))
print('  total:',total,'correct:',correct)

In [ ]:
# Guardar la salida

if(PAD):
    output=np.reshape(output,(-1,H1))
    output=output[sizey//2:V1-sizey//2,sizex//2:H1-sizex//2]
    H1=H1-2*(sizex//2); V1=V1-2*(sizey//2)
    output=np.reshape(output,(H1*V1))

save_pgm(output,H1,V1,nclases,'../results/predictions/predictions_cnn21_onnx.pgm')

In [ ]:
# Mostrar la salida

OUTPUT='../results/predictions/predictions_cnn21_onnx.pgm'

(imagen_output, H1, V1) = read_pgm(OUTPUT)

# Convertir la lista a array y redimensionar
imagen_output = np.array(imagen_output, dtype=np.uint8).reshape(V1, H1)

# Mostrar la imagen
plt.imshow(imagen_output)
plt.show()